In [ ]:
import tensorflow as tf
#準備資料集
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84140032/84125825 [==============================] - 2s 0us/step


In [ ]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [ ]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    #取資料,使用glob抓取可避免資料夾路徑有誤
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [ ]:
train_df = getdata("train")
test_df = getdata("test")

In [ ]:
test_df

,content,sentiment
0,"Valentine ""Dogkiller"" Dussaut and Joe ""The Jud...",1
1,Ruthless mercenary Bruno Rivera (Paul Naschy i...,1
2,Should we take the opening shot as a strange f...,1
3,"Being a retired medical/health field ""toiler i...",1
4,I wasn't expecting a great deal from this film...,1
...,...,...
24995,"I give it a 2, because of the beautiful Medite...",0
24996,How bad can you make a film. A good question w...,0
24997,"Oh, my. Oh, this is a *really* bad movie. The ...",0
24998,Apparently re-cut episodes from the Gangbuster...,0


In [ ]:
# Tokenize: 把你的詞變成數字
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000) #只出現頻率最高的前3000個字
tok.fit_on_texts(train_df["content"])

In [ ]:
# tok.word_index
# tok.index_word

In [ ]:
# Sequence: 化成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])

In [ ]:
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,372,17,13,1,330,19,5,320,5.0,1706.0,4.0,2.0,1.0,43.0,23.0,147.0,33.0,409.0,8.0,3.0,1425.0,264.0,15.0,768.0,5.0,600.0,50.0,2.0,40.0,41.0,29.0,248.0,18.0,6.0,1.0,1890.0,28.0,26.0,44.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45,256,6,924,8,3,1036,4,11.0,19.0,588.0,329.0,1.0,1321.0,4.0,11.0,2781.0,83.0,31.0,20.0,1085.0,1.0,90.0,15.0,245.0,1552.0,304.0,979.0,6.0,3.0,751.0,5.0,1.0,956.0,3.0,510.0,206.0,979.0,16.0,823.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45,222,28,509,12,204,112,74,3.0,334.0,4.0,42.0,1.0,207.0,1058.0,16.0,2554.0,1614.0,117.0,136.0,2.0,29.0,184.0,23.0,217.0,112.0,221.0,205.0,57.0,8.0,1.0,953.0,4.0,1.0,368.0,1016.0,904.0,37.0,1588.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,129,3,1574,1433,4,1,831,1519.0,26.0,112.0,1.0,1519.0,66.0,193.0,822.0,31.0,7.0,7.0,3.0,129.0,2718.0,24.0,505.0,20.0,1.0,18.0,302.0,4.0,1927.0,139.0,3.0,1276.0,411.0,59.0,2032.0,26.0,139.0,12.0,931.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,852,7,7,22,212,21,532,16,202.0,99.0,118.0,1.0,44.0,5.0,32.0,2196.0,4.0,1293.0,109.0,722.0,8.0,1.0,1768.0,18.0,30.0,2119.0,2.0,8.0,2863.0,258.0,51.0,6.0,937.0,7.0,7.0,6.0,54.0,1395.0,18.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1,1118,15,737,5,27,3,478,28.0,12.0,6.0,1209.0,36.0,1.0,618.0,1967.0,9.0,1023.0,5.0,2040.0,225.0,16.0,3.0,62.0,1.0,2523.0,2.0,838.0,12.0,325.0,81.0,68.0,914.0,5.0,30.0,1.0,55.0,1.0,19.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,343,19,1,84,1372,1643,1,265,16.0,35.0,73.0,2318.0,2.0,12.0,28.0,5.0,27.0,2640.0,242.0,31.0,157.0,399.0,19.0,40.0,37.0,3.0,1185.0,5.0,1.0,1882.0,1414.0,25.0,11.0,14.0,109.0,1.0,19.0,12.0,1720.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,44,5,27,31,227,1,1554,246.0,17.0,10.0,25.0,107.0,8.0,1.0,233.0,888.0,150.0,51.0,10.0,216.0,12.0,485.0,13.0,8.0,9.0,10.0,2627.0,9.0,423.0,27.0,96.0,75.0,3.0,17.0,234.0,26.0,44.0,74.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,10,25,107,108,49,99,583,2,99.0,16.0,18.0,11.0,28.0,1873.0,1.0,164.0,183.0,5.0,27.0,3.0,34.0,571.0,5.0,76.0,291.0,5.0,2234.0,46.0,2586.0,1.0,17.0,1502.0,98.0,429.0,4.0,718.0,1104.0,2.0,6.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
INPUT_LENGTH = 512
INPUT_DIM = 3000
OUTPUT_DIM = 128

In [ ]:
# Padding: 截長補短變成一樣長
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=INPUT_LENGTH)
x_test_pad = pad_sequences(x_test_seq, maxlen=INPUT_LENGTH)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,301,117,1,15,1,751,26,1983,46,581,51,26,119,46,154,20,53,8,160,18,21,14,49,14,1,83,19,18,2,448,77,354,9,1474,31,324,1301,7,7,1174
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2425,34,53,147,2,171,466,1,62,8,3,2588,4,5,709,2006,53,1883,111,2,57,1492,3,618,16,1016,574,2095,7,7,924,8,257,3,1036,898,5,69,130,2353
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,104,1906,15,1848,3,752,12,1025,5,987,5,196,91,872,2,1,2064,9,213,731,55,2,141,27,821,9,77,590,148,47,431,23,54,286,369,50,4,95,786,155
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,881,2888,1,129,122,2,414,80,3,191,4,1,127,7,7,42,35,12,9,817,627,53,109,3,173,4,250,463,3,1015,274,1500,3,285,763,6,3,52,315,17
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,88,4,9,49,1153,718,92,45,54,14,2473,1038,41,157,19,42,1,240,4,17,22,67,866,142,2,354,14,193,14,22,89,94,1,1318,4,533,96,73,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,31,1338,89,614,866,70,372,5,1,687,7,7,1209,36,1,336,860,1,113,215,1,115,342,405,3,1498,236,14,124,1,4,1,174,441,3,1386,7,7,339,155
24996,9,66,282,36,1,1,776,4,1,1372,1,585,4,32,2,12,474,133,12,163,22,920,12,131,1907,380,967,472,9,13,29,15,91,55,36,130,5,172,1006,1722,...,283,15,256,2,1,474,1171,2081,12,1,4,9,66,122,231,100,1,19,645,9,13,18,21,1,84,1372,44,413,91,831,8,1,55,4,434,7,7,1239,43,4
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,234,26,44,74,8,46,181,539,105,2,26,13,3,181,1254,281,352,54,28,141,435,65,55,20,11,19,10,700,140,4,9,2,10,89,232,12,10,1046,3,152
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1386,8,257,706,385,29,11,44,6,798,136,16,1,16,908,321,136,2,798,50,538,2,564,533,12,163,9,52,1330,17,2,10,2304,383,29,1,99,4,11,429


In [ ]:
#第一種模型(標準模型) > 參數量太多
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
layers = [
    # 3001(種詞) * 128(個情緒)
    Embedding(INPUT_DIM+1, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 128)          384128    
                                                                 
 flatten (Flatten)           (None, 65536)             0         
                                                                 
 dense (Dense)               (None, 256)               16777472  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 17,162,114
Trainable params: 17,162,114
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#第二種模型 > 使用GAP,參數量較少
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
layers = [
    # 3001(種詞) * 128(個情緒)
    Embedding(INPUT_DIM+1, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 512, 128)          384128    
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 一個輸出(二元分類): BinaryCrossEntropy p log 1/q + (1 - p) log 1/1-q
# 多個輸出(多元分類): CategoricalCrossEntropy pi log1/qi
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              # "adam"也可以
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [ ]:
# batch_size: 看多少筆, 做一次梯度下降(幾10~幾100)
# epochs: 所有資料看幾輪(負責結束訓練)
# batch_size=200
# 一epochs: 54000 / 200 = 270(次梯度下降)
# verbose=0(quiet) 1(default) 2(no bar)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("imdb.h5", save_best_only=True)
]
model.fit(x_train_pad,
          y_train,
          batch_size=200,
          epochs=100,
          validation_split=0.1,
          verbose=2,
          callbacks=callbacks)

Epoch 1/100
113/113 - 4s - loss: 0.6318 - accuracy: 0.6452 - val_loss: 0.7020 - val_accuracy: 0.5252 - 4s/epoch - 33ms/step
Epoch 2/100
113/113 - 1s - loss: 0.4645 - accuracy: 0.8252 - val_loss: 0.4668 - val_accuracy: 0.7968 - 1s/epoch - 11ms/step
Epoch 3/100
113/113 - 1s - loss: 0.3599 - accuracy: 0.8682 - val_loss: 0.4091 - val_accuracy: 0.8204 - 1s/epoch - 11ms/step
Epoch 4/100
113/113 - 1s - loss: 0.3095 - accuracy: 0.8814 - val_loss: 0.4249 - val_accuracy: 0.8076 - 1s/epoch - 11ms/step
Epoch 5/100
113/113 - 1s - loss: 0.2800 - accuracy: 0.8925 - val_loss: 0.3796 - val_accuracy: 0.8308 - 1s/epoch - 11ms/step
Epoch 6/100
113/113 - 1s - loss: 0.2603 - accuracy: 0.9004 - val_loss: 0.4076 - val_accuracy: 0.8180 - 1s/epoch - 11ms/step
Epoch 7/100
113/113 - 1s - loss: 0.2462 - accuracy: 0.9051 - val_loss: 0.3592 - val_accuracy: 0.8436 - 1s/epoch - 11ms/step
Epoch 8/100
113/113 - 1s - loss: 0.2353 - accuracy: 0.9100 - val_loss: 0.3476 - val_accuracy: 0.8556 - 1s/epoch - 11ms/step
Epoch 9/

In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 2s 3ms/step - loss: 0.2894 - accuracy: 0.8820


[0.28943106532096863, 0.8820000290870667]

In [ ]:
l = [
    Embedding(INPUT_DIM+1, OUTPUT_DIM, mask_zero=True)
]
remain = model.layers[1:]
model_use = Sequential(l+remain)
model_use.layers[0].set_weights(model.layers[0].get_weights())
model_use.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         384128    
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
review = input("影評:")
review_seq = tok.texts_to_sequences([review])
proba = model_use.predict(review_seq)[0]
trans = ["neg", "pos"]
for p, sentiment in zip(proba, trans):
    print(sentiment, ":", p)

影評:movie
neg : 0.54052734
pos : 0.45947263


In [ ]:
# 時間序列組合
# RNN(短期記憶) -> 改進 LSTM, GRU(長期記憶)
# 因為不可平行化>時間久,不可使用GPU >>> 被Bert取代
from tensorflow.keras.layers import SimpleRNN
layers = [
    # 3001(種詞) * 128(個情緒)
    Embedding(INPUT_DIM+1, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
    # Embedding W: 128 * 64 + 記憶 W: 64 * 64 + bias: 64 = 12352
    SimpleRNN(64),
    Dense(2, activation="softmax")
]
rnn = Sequential(layers)
rnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 512, 128)          384128    
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 396,610
Trainable params: 396,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
l = [
     Embedding(INPUT_DIM+1, OUTPUT_DIM),
     GlobalAveragePooling1D()
]
partial = Sequential(l)
partial.layers[0].set_weights(model.layers[0].get_weights())
partial.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 128)         384128    
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from scipy.spatial.distance import cosine
# 使用colab forms,改參數比較方便
w1 = "This movie sucks"#@param {type:"string"}
w2 = "I hate this movie. It's totally a disaster."#@param {type:"string"}
seq1 = tok.texts_to_sequences([w1])
seq2 = tok.texts_to_sequences([w2])
em1 = partial.predict(seq1)[0]
em2 = partial.predict(seq2)[0]
dis = 1 - cosine(em1, em2)
print("cos相似度:", dis)

cos相似度: 0.9605348110198975
